Merry Christmas everyone! 

[Advent of Code day 7](http://adventofcode.com/2018/day/7)

## Part 1

Probably easiest to create a `dict` of the prerequisites, and then choose the next alphabetical step which doesn't have any outstanding prerequisites.

In [1]:
import re

Now create an re that matches against any of the steps:

In [2]:
step_re=re.compile('Step (\w) must be finished before step (\w) can begin.')

In [3]:
test_input_str='''
Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.
'''

In [4]:
input_str=test_input_str

We should be able to use a `.findall` to identify each of the steps:

In [5]:
step_re.findall(input_str)

[('C', 'A'),
 ('C', 'F'),
 ('A', 'B'),
 ('A', 'D'),
 ('B', 'E'),
 ('D', 'E'),
 ('F', 'E')]

And add to a dictionary of prerequisites:

In [6]:
prereq_dict={}

for (dependency, step) in step_re.findall(input_str):

    # Minor hack to ensure that all steps are contained:
    if dependency not in prereq_dict:
        prereq_dict[dependency]=set()
    if step not in prereq_dict:
        prereq_dict[step]=set()
    
    # Add the actual dependency
    prereq_dict[step].add(dependency)

prereq_dict

{'A': {'C'},
 'B': {'A'},
 'C': set(),
 'D': {'A'},
 'E': {'B', 'D', 'F'},
 'F': {'C'}}

Loop through the dict. At each stage, carry out the step which is first alphabetically of those which have no remaining prerequisites.

In [7]:
out=''

while prereq_dict:
    # Get next step
    next_step=sorted([k for k in prereq_dict if not prereq_dict[k]])[0]

    # Remove the next step from the queue:
    prereq_dict.pop(next_step)

    # Remove from prerequisites
    for k in prereq_dict:
        if next_step in prereq_dict[k]:
            prereq_dict[k].remove(next_step)
            
    # Add to the output
    out+=next_step

In [8]:
assert out=='CABDFE'

Should be OK. Try for the puzzle input.

In [9]:
prereq_dict={}

for (dependency, step) in step_re.findall(open('inputs/day7').read()):

    # Minor hack to ensure that all steps are contained:
    if dependency not in prereq_dict:
        prereq_dict[dependency]=set()
    if step not in prereq_dict:
        prereq_dict[step]=set()
    
    # Add the actual dependency
    prereq_dict[step].add(dependency)

out=''

while prereq_dict:
    # Get next step
    next_step=sorted([k for k in prereq_dict if not prereq_dict[k]])[0]

    # Remove the next step from the queue:
    prereq_dict.pop(next_step)

    # Remove from prerequisites
    for k in prereq_dict:
        if next_step in prereq_dict[k]:
            prereq_dict[k].remove(next_step)
            
    # Add to the output
    out+=next_step
    
out


'GKCNPTVHIRYDUJMSXFBQLOAEWZ'

## Part 2

I don't know whether this is a search task. Let's start on the assumption that we can just work on the tasks in alphabetical order again.

At each stage, there are a number of options:

1. Assign a worker to a task
2. Work on the task for one second
3. Take a worker off a completed task

Let's say that the state at any given time consists of:

1. The current time
2. The available workers
3. The ongoing tasks, with
    - the task ID
    - the time remaining on that task
    - the worker working on that task
4. The dictionary of prerequisites

Much as I hate OO programming, and even more python's horrible excuse for objects, I hate inplace functions even more, so let's create an object >:-(. This is where I'd go over to a sensible language if this was a serious task.

In [10]:
class State():
    
    def __init__(self,
                 input_str,
                 time_offset=60,
                 workers=set([1,2,3,4,5])):
        step_re=re.compile('Step (\w) must be finished before step (\w) can begin.')
        
        for (dependency, step) in step_re.findall(input_str):
            # Minor hack to ensure that all steps are contained:
            if dependency not in prereq_dict:
                prereq_dict[dependency]=set()
            if step not in prereq_dict:
                prereq_dict[step]=set()

            # Add the actual dependency
            prereq_dict[step].add(dependency)
            
        self.my_prereqs=prereq_dict
            
        self.my_time_offset=time_offset
        self.my_available_workers=workers
        self.my_time=0
        self.my_ongoing_tasks=[]

        self.my_completed_tasks=[]
        return None

    def complete(self):
        if self.my_prereqs or self.my_ongoing_tasks:
            return False
        else:
            return True

    def to_dict(self):
        'For ease of viewing'
        return {'time':self.my_time,
                'schedule':self.my_prereqs,
                'available workers':self.my_available_workers,
                'ongoing tasks':self.my_ongoing_tasks,
                'completed tasks':self.my_completed_tasks}
    
    def assign_task(self):
        '''Assigns a worker to the next task. Return False if no
           available tasks or workers, True otherwise.'''
            # If no available workers, return None
        if not self.my_available_workers:
            return False
        # Find next available task:
        next_tasks=sorted([k for k in self.my_prereqs if not self.my_prereqs[k]])
        if not next_tasks:
            return False
        else:
            next_task=next_tasks[0]

        # Add the next task with an assigned worker to the ongoing tasks
        self.my_prereqs.pop(next_task)
        assigned_worker=self.my_available_workers.pop()
        self.my_ongoing_tasks.append({'task':next_task,
                                      'assigned worker':assigned_worker,
                                      'time remaining':self.my_time_offset + ord(next_task) -ord('A')+1})

        return True
    
    def tick(self):
        '''Advance the state by one second'''
        # First, need to reduce the required times in the
        # ongoing tasks by 1 second
        self.my_time+=1
        for t in self.my_ongoing_tasks:
            t['time remaining'] -= 1
            
        # Then for any completed tasks, need to
        # remove the task from the list, release
        # the worker, complete any prereq. info, and
        # add the completed task to the output.
        for t in self.my_ongoing_tasks:
            if t['time remaining']==0:
                self.my_available_workers.add(t['assigned worker'])
                self.my_completed_tasks.append(t['task'])
                self.my_ongoing_tasks.remove(t)
                for p in self.my_prereqs:
                    if t['task'] in self.my_prereqs[p]:
                        self.my_prereqs[p].remove(t['task'])
        return True


Good, now check with five workers, the given offset and the puzzle input:

In [11]:
s=State(open('inputs/day7').read(), workers={1, 2, 3, 4, 5}, time_offset=60)

while not s.complete():
    if s.assign_task():
        pass
    else:
        s.tick()

s.to_dict()['time']

1265